In [2]:
# YOU MUST USE 3.6.0 AND ABOVE FOR RUNNING GLMNET
#install.packages("psycho")
#install.packages("zoo")
#install.packages("scales")
#install.packages("rlang")
#install.packages("tidyverse")

#install.packages('lm.beta')
#install.packages("standardize")
library(data.table)
library("rvest")

library("zoo")
library("dplyr")

library(tidyverse)
library(caret)
library("lm.beta")
library("psycho")

library("standardize")
library("glmnet")
#install.packages("doParallel")
library("doParallel")
#install.packages("foreach")
#install.packages("pROC")
library("foreach")
library("pROC")

        
        
      #  rm(list=ls())
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2219942,118.6,3651298,195.1,3651298,195.1
Vcells,3698803,28.3,8388608,64.0,5927122,45.3


In [3]:
start_time <- Sys.time()
new_data=fread("num_slim_data_5.csv")

end_time <- Sys.time()

end_time - start_time
dim(new_data)

Time difference of 11.46325 secs

[1] 478174    165

In [4]:
grep("count",colnames(new_data))

[1] 165

In [5]:
scale=as.data.frame(scale(new_data))
scale_data=scale
dim(scale_data)




[1] 478174    165

In [6]:
coeff=NULL
sum=NULL
model=NULL
r=NULL
beta=NULL
for (i in 1:165){
   
   model[[i]] <- lm(data=new_data,formula = count ~ new_data[[i]] )
   coeff[[i]]=model$coefficients
   sum[[i]]=summary(model[[i]])
    r[[i]]=sum[[i]]$adj.r.squared
    beta[[i]]=lm.beta(model[[i]])
}






Warning message in summary.lm(model[[i]]):
“essentially perfect fit: summary may be unreliable”

In [7]:
new_beta=NULL
for (i in 1:165){
    new_beta[i]=abs(beta[[i]]$`standardized.coefficients`[2])
}

In [8]:
registerDoParallel(cores = 4)

In [26]:
set.seed(123)
new_new=scale_data[sample(1:nrow(new_data),50000,replace = F),]

In [27]:
set.seed(123)
new_new=scale_data[sample(1:nrow(new_data),50000,replace = F),]
n=nrow(new_new)
sample <- sample(seq(n), size = n * 0.5, replace = FALSE)
training.samples <- new_new$count %>%
  createDataPartition(p = 0.8, list = FALSE)
train.data  <- new_new[training.samples, ]
test.data <- new_new[-training.samples, ]
# Predictor variables
x <- model.matrix(count~., train.data)[,-1]
# Outcome variable
y <- train.data$count
dim(new_new)

[1] 50000   165

In [32]:
dim(x)

[1] 40002   164

In [13]:
print(y[1:20])

 [1] 6.77 7.90 5.80 5.60 7.50 6.40 6.23 7.29 7.10 8.26 8.15 5.40 7.83 8.14 8.52
[16] 5.60 5.15 7.57 6.96 7.11


In [14]:
lasso=glmnet(x, y, alpha = 1, lambda = NULL)

In [15]:
ridge=glmnet(x, y, alpha = 0, lambda = NULL)

In [16]:
set.seed(123) 
cv.ridge <- cv.glmnet(x, y, alpha = 0)
# Display the best lambda value
cv.ridge$lambda.min

[1] 0.2853351

In [17]:
# Fit the final model on the training data
model.ridge <- glmnet(x, y, alpha = 0, lambda = cv.ridge$lambda.min)
# Display regression coefficients
coef(model.ridge)

165 x 1 sparse Matrix of class "dgCMatrix"
                        s0
(Intercept)  -1.595788e+01
eid           4.778657e-09
`34-0.0`      1.021243e-02
`46-0.0`     -5.129184e-04
`47-0.0`      6.198229e-03
`48-0.0`      1.336496e-02
`49-0.0`     -7.445437e-03
`50-0.0`     -6.636036e-03
`51-0.0`     -3.410307e-03
`92-0.0`      9.567104e-06
`96-0.0`     -4.318153e-04
`102-0.0`     2.178183e-02
`129-0.0`    -2.503078e-07
`130-0.0`    -1.571772e-07
`135-0.0`     1.274495e-02
`137-0.0`     4.103380e-02
`189-0.0`     1.277037e-02
`699-0.0`    -8.989530e-04
`709-0.0`    -2.378723e-02
`874-0.0`     2.044747e-04
`894-0.0`     5.367497e-04
`1269-0.0`    1.477252e-02
`1279-0.0`   -8.856087e-03
`1289-0.0`    4.149032e-03
`1299-0.0`   -2.514401e-04
`1309-0.0`   -2.943549e-02
`1319-0.0`    2.464657e-03
`1438-0.0`   -4.349984e-04
`1458-0.0`   -2.694106e-02
`1488-0.0`    2.512856e-04
`1528-0.0`   -4.104731e-03
`1737-0.0`   -6.652890e-03
`1807-0.0`   -3.317091e-03
`2139-0.0`   -5.748219e-03
`2149-0.0`  

In [18]:
# Make predictions on the test data
x.test <- model.matrix(count ~., test.data)[,-1]
predictions <- model.ridge %>% predict(x.test) %>% as.vector()
# Model performance metrics
data.frame(
  RMSE = RMSE(predictions, test.data$count),
  Rsquare = R2(predictions, test.data$count)
)

RMSE,Rsquare
<dbl>,<dbl>
2.222388,0.07850877


#### Note that by default, the function glmnet() standardizes variables so that their scales are comparable. However, the coefficients are always returned on the original scale.



In [39]:
# Find the best lambda using cross-validation
set.seed(123) 
cv.lasso <- cv.glmnet(x, y, alpha = 1)
# Display the best lambda value
cv.lasso$lambda.min

[1] 0.01385049

In [40]:
# Fit the final model on the training data
model.lasso <- glmnet(x, y, alpha = 1, lambda = cv.lasso$lambda.min)
# Dsiplay regression coefficients
coef(model.lasso)

165 x 1 sparse Matrix of class "dgCMatrix"
                        s0
(Intercept)  -5.801222e+01
eid           .           
`34-0.0`      2.955894e-02
`46-0.0`      .           
`47-0.0`      .           
`48-0.0`      2.136160e-02
`49-0.0`     -1.775000e-03
`50-0.0`     -1.455543e-02
`51-0.0`      .           
`92-0.0`      .           
`96-0.0`     -2.303063e-04
`102-0.0`     2.337164e-02
`129-0.0`    -1.801573e-07
`130-0.0`     .           
`135-0.0`     .           
`137-0.0`     7.242607e-02
`189-0.0`     1.004008e-02
`699-0.0`     .           
`709-0.0`    -2.862142e-02
`874-0.0`     .           
`894-0.0`     7.806112e-04
`1269-0.0`    8.717369e-03
`1279-0.0`   -4.774959e-03
`1289-0.0`    2.296595e-03
`1299-0.0`    .           
`1309-0.0`   -2.668471e-02
`1319-0.0`   -5.786788e-04
`1438-0.0`   -7.425872e-04
`1458-0.0`   -2.418928e-02
`1488-0.0`    .           
`1528-0.0`    .           
`1737-0.0`   -2.906100e-03
`1807-0.0`   -2.172676e-03
`2139-0.0`   -3.202479e-03
`2149-0.0`  

In [41]:
# Make predictions on the test data
x.test <- model.matrix(count ~., test.data)[,-1]
predictions <- model.lasso %>% predict(x.test) %>% as.vector()
# Model performance metrics
data.frame(
  RMSE = RMSE(predictions, test.data$count),
  Rsquare = R2(predictions, test.data$count)
)

RMSE,Rsquare
<dbl>,<dbl>
2.211446,0.06391145


In [23]:
ls()

[1] "cv.lasso"         "cv.ridge"         "end_time"         "lasso"           
 [5] "model.lasso"      "model.ridge"      "n"                "new_data"        
 [9] "new_new"          "predictions"      "ridge"            "sample"          
[13] "scale"            "scale_data"       "start_time"       "test.data"       
[17] "train.data"       "training.samples" "x"                "x.test"          
[21] "y"

In [96]:
set.seed(123)
new_new=scale_data[sample(1:nrow(new_data),100000,replace = F),]
n=nrow(new_new)
sample <- sample(seq(n), size = n * 0.5, replace = FALSE)
training.samples <- new_new$count %>%
  createDataPartition(p = 0.8, list = FALSE)
train.data  <- new_new[training.samples, ]
test.data <- new_new[-training.samples, ]
# Predictor variables
x <- model.matrix(count~., train.data)[,-1]
# Outcome variable
y <- train.data$count
dim(new_new)

[1] 100000    165

In [97]:
# Build the model using the training set
set.seed(123)
model.ela <- train(
  count ~., data = train.data, method = "glmnet",
  trControl = trainControl("cv", number = 10),
  tuneLength = 10
)
# Best tuning parameter
model.ela$bestTune

,alpha,lambda
,<dbl>,<dbl>
4,0.1,0.0009176004


In [98]:
model.best.ela <- glmnet(x, y, alpha = 0.7, lambda = 0.01219148)
# Dsiplay regression coefficients
coef(model.best.ela)

165 x 1 sparse Matrix of class "dgCMatrix"
                        s0
(Intercept)  -0.0016660513
eid           .           
`34-0.0`      0.0443990182
`46-0.0`      .           
`47-0.0`      .           
`48-0.0`      0.1105600671
`49-0.0`      .           
`50-0.0`     -0.0802554421
`51-0.0`     -0.0190970154
`92-0.0`      .           
`96-0.0`     -0.0043057965
`102-0.0`     0.1153646868
`129-0.0`     .           
`130-0.0`     .           
`135-0.0`     0.0150186343
`137-0.0`     0.0581277402
`189-0.0`     0.0132502420
`699-0.0`    -0.0006489901
`709-0.0`     .           
`874-0.0`     .           
`894-0.0`     .           
`1269-0.0`    .           
`1279-0.0`    .           
`1289-0.0`    .           
`1299-0.0`    .           
`1309-0.0`   -0.0174025631
`1319-0.0`    .           
`1438-0.0`    .           
`1458-0.0`   -0.0130447397
`1488-0.0`    0.0023274954
`1528-0.0`    .           
`1737-0.0`    .           
`1807-0.0`   -0.0029545093
`2139-0.0`   -0.0099731758
`2149-0.0`  

In [93]:


x.test <- model.matrix(count ~., test.data)[,-1]
predictions <- model.best.ela %>% predict(x.test) %>% as.vector()
# Model performance metrics
data.frame(
  RMSE = RMSE(predictions, test.data$count),
  Rsquare = R2(predictions, test.data$count)
)

RMSE,Rsquare
<dbl>,<dbl>
1.045114,0.08130832


In [94]:
beta_net_ela=cbind(as.data.frame(x=colnames(x)),as.data.frame(x=as.vector(model.best.ela$beta)))
colnames(beta_net_ela)=c("variable","beta")

In [95]:
beta_net_ela[beta_net_ela$beta>0,]

,variable,beta
,<fct>,<dbl>
2,`34-0.0`,0.0858030355
4,`47-0.0`,0.0098399683
5,`48-0.0`,0.1094198569
11,`102-0.0`,0.1291060930
15,`137-0.0`,0.0539639680
16,`189-0.0`,0.0147246311
20,`894-0.0`,0.0128977267
21,`1269-0.0`,0.0230086846
34,`2149-0.0`,0.0142767115


In [87]:
beta_net_ela[beta_net_ela$beta>0,]

,variable,beta
,<fct>,<dbl>
2,`34-0.0`,1.009776e-02
5,`48-0.0`,1.665199e-02
11,`102-0.0`,2.351627e-02
15,`137-0.0`,5.694711e-02
16,`189-0.0`,1.260256e-02
29,`1488-0.0`,3.799752e-03
35,`2217-0.0`,2.680677e-04
37,`3061-0.0`,1.503311e-03
42,`4080-0.0`,2.603806e-03


1. Year of birth
2. Waist circumference
3. Pulse rate
4. Number of treatments/medications taken
5. Townsend deprivation index at recruitment
6. Forced expiratory volume in 1-second (FEV1) Z-score
7. FEV1/ FVC ratio Z-score
8. Arm fat mass (right)
9. Income score (England)
10. Education score (England)
11. !! Haematocrit percentage
12. !! Red blood cell (erythrocyte) distribution width

In [51]:
dim(scale_data)

[1] 478174    165

In [106]:
grep("30070-0.0",colnames(scale_data))

[1] 132

In [6]:
scale_data[1:20,c(128:132)]

30030-0.0,30040-0.0,30050-0.0,30060-0.0,30070-0.0
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
-1.4018767,-0.53406873,-0.630190654,-0.37438776,0.28021224
0.2291841,-0.02605847,-0.182638748,-0.29074653,-0.49935561
0.5638328,0.81411235,0.613587319,-0.11417060,0.08785134
-0.2404489,0.35603472,0.004708562,-0.56025716,-0.94482295
1.4384189,-1.04642096,-0.494884264,0.73153517,-1.00556849
-0.2207637,0.79891546,0.858179640,0.35979637,-1.20805365
0.6032032,0.21709174,0.145219044,-0.06770325,-0.23612490
0.2291841,-0.89445208,-0.494884264,0.45273107,-1.20805365
0.7409998,0.13242337,0.514709572,0.79658946,-0.43861006


In [12]:
new_data[1:20,c(127:132)]

30020-0.0,30030-0.0,30040-0.0,30050-0.0,30060-0.0,30070-0.0
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
12.31,36.10,88.66,30.24,34.11,13.77
14.30,41.90,91.00,31.10,34.20,13.00
14.82,43.09,94.87,32.63,34.39,13.58
13.64,40.23,92.76,31.46,33.91,12.56
16.30,46.20,86.30,30.50,35.30,12.50
14.10,40.30,94.80,33.10,34.90,12.30
14.89,43.23,92.12,31.73,34.44,13.26
14.70,41.90,87.00,30.50,35.00,12.30
15.46,43.72,91.73,32.44,35.37,13.06


In [13]:
new_new= new_data[,-c(127:132)]

In [14]:
new_new_scale= scale_data[,-c(127:132)]

In [15]:
dim(new_new)

[1] 478174    159

In [16]:
dim(new_new_scale)

[1] 478174    159

In [17]:
fwrite(new_new,"num_slim_data6.csv")

In [18]:
fwrite(new_new_scale,"num_slim_data_scale6.csv")